In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [33]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.identity_mapping = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.identity_mapping = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        identity = self.identity_mapping(x)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = x + identity
        x = self.relu(x)
        return x

In [34]:
class ResNet34(nn.Module):
    def __init__(self, num_classes=200):
        super(ResNet34, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.layer1 = self.make_layer(in_channels=64, out_channels=64, num_blocks=3, stride=1)
        self.layer2 = self.make_layer(in_channels=64, out_channels=128, num_blocks=4, stride=2)
        self.layer3 = self.make_layer(in_channels=128, out_channels=256, num_blocks=6, stride=2)
        self.layer4 = self.make_layer(in_channels=256, out_channels=512, num_blocks=3, stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(Block(in_channels, out_channels, stride))  # Downsampling block
        for _ in range(1, num_blocks):
            layers.append(Block(out_channels, out_channels))  # Identity blocks
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

In [4]:
data_dir = "tiny-imagenet-200/train"
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),      
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root=data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

print(f"Dataset size: {len(train_dataset)}")

Dataset size: 100000


In [5]:
# hyperparameters
n_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Current device for training:", device)

model = ResNet34(num_classes=200).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

model.to(device)
all_loss = []

for epoch in range(n_epochs):
    model.train()
    
    for idx, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
    
        all_loss.append(loss.item())
    
        if idx % 100 == 0:
            print(f"Epoch {epoch+1}/{n_epochs} - Step {idx}: Loss {loss.item():4f}")

Current device for training: cuda
Epoch 1/30 - Step 0: Loss 5.532509
Epoch 1/30 - Step 100: Loss 5.164623
Epoch 1/30 - Step 200: Loss 5.195043
Epoch 1/30 - Step 300: Loss 5.177605
Epoch 1/30 - Step 400: Loss 5.120026
Epoch 1/30 - Step 500: Loss 4.661223
Epoch 1/30 - Step 600: Loss 4.664922
Epoch 1/30 - Step 700: Loss 4.525347
Epoch 1/30 - Step 800: Loss 4.611470
Epoch 1/30 - Step 900: Loss 4.471470
Epoch 1/30 - Step 1000: Loss 4.500185
Epoch 1/30 - Step 1100: Loss 4.367681
Epoch 1/30 - Step 1200: Loss 4.338236
Epoch 1/30 - Step 1300: Loss 4.171047
Epoch 1/30 - Step 1400: Loss 4.339691
Epoch 1/30 - Step 1500: Loss 4.294849
Epoch 2/30 - Step 0: Loss 4.122612
Epoch 2/30 - Step 100: Loss 3.936005
Epoch 2/30 - Step 200: Loss 4.280327
Epoch 2/30 - Step 300: Loss 3.922568
Epoch 2/30 - Step 400: Loss 3.988447
Epoch 2/30 - Step 500: Loss 3.958541
Epoch 2/30 - Step 600: Loss 3.903003
Epoch 2/30 - Step 700: Loss 3.587006
Epoch 2/30 - Step 800: Loss 3.621310
Epoch 2/30 - Step 900: Loss 3.255205
Ep

KeyboardInterrupt: 

In [6]:
torch.save(model.state_dict(), "models/res_net/res_net_1.pth")

In [29]:
data_dir = "tiny-imagenet-200/val"
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),      
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = ImageFolder(root=data_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True, num_workers=4)

In [37]:
model = ResNet34()
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        image, label = image.to(device), label.to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

        correct += (predicted == label).sum().item()
        total += label.size(0)

accuracy = 100 * correct / total

print(accuracy)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [36]:
model = ResNet34().to(device)
model.eval()

for image, label in test_loader:
    image, label = image.to(device), label.to(device)
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)

    print(predicted)
    print(label)
    break

tensor([ 35,  35, 134, 134,  35,  35, 134, 133, 134, 134,  35, 134, 134,  35,
         35,  35,  35,  35,  35,  35, 134,  35, 134, 134,  35, 164, 134,  35,
         35,  35,  35, 134,  35,  35,  35,  35,  35, 134,  35,  35, 134, 134,
         35,  35, 134,  35, 134,  35, 164,  35,  35,  35, 164, 134,  35,  35,
         35, 134,  35,  35,  35, 134, 134, 134, 134,  35,  35,  35, 134, 134,
         35,  35, 134,  35,  35, 134, 134,  35,  35,  35,  35,  35,  35,  35,
         35, 134, 134,  35, 134,  35, 134, 134, 134,  35,  35, 134, 134, 134,
        134,  35], device='cuda:0')
tensor([ 74,  30, 154, 174,  45, 139, 152,  34, 174, 116,  34, 119, 193,  48,
         85,  46, 199,  85,  46, 142, 114,  28,  65, 111,  84, 196, 102, 162,
         22, 141,  78, 174,  60, 191, 153,  27,  73, 108,  56,  71, 193, 148,
        196,  50, 115,  13,  53, 197, 133,  60, 114, 138,  94, 164, 196,  23,
        157, 126,  54,  68, 120,  40, 199, 164, 122, 123, 186, 198, 149,  44,
        199, 197, 179, 183, 